## Matrix Economy

### 1. Function of X,Y that does not exceed O(N)

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

# want only matrices of Nxk or Nxm (or smaller)

def proj_fun(X,Y):
    
    first_half = X@inv(X.T@X) # N x k
    second_half = X.T@Y # k x m so tiny
    
    return(first_half@second_half)

### 2. Factory function of X to get $P_x$ 

In [ ]:
# define a function of x that returns a function of y 
def makeproj(X):
    def multproj(Y):
        first_half = X@inv(X.T@X) # N x k
        second_half = X.T@Y # k x m so tiny
        return(first_half@second_half)
    return multproj

# to test it
#testx = np.array([[1,0],[0,1],[1,0]])
#testy = np.array([0,0,1])

#test_mult = makeproj(testx)
#test_mult(testy)

### 3. Get diagonal of P but do not exceed O(N)

In [ ]:
def diag_fun(X):
    
    N = X.shape[0]
    first_half = X@inv(X.T@X) # N x k
    
    diag_vec = np.empty([N, 1])
    
    for i in range(N):
        diag_vec[i] = first_half[i,:]@X.T[:,i] # diagonal elements are dot product of ith row and ith column
        #I think this is O(N)? but not sure
        
    return(diag_vec)

### 4. Dask library to construct larger matrices

In [ ]:
import dask.array as da

def proj_dask_fun(X,Y):
    
    N = X.shape[0]
    denom = 1 # set this based on N 
    
    P = da.from_array(X@inv(X.T@X)@X.T, chunks = (N/denom,N/denom)) 
    # we can make this a dask array even though it's big; chunk it based on N (not sure best practice here)
    
    proj = P@Y
    # multiply as usual
    
    #return using compute() to return numpy array
    return(proj.compute())

# to test
#testx = np.array([[1,0],[0,1],[1,0]])
#testy = np.array([0,0,1])

#test = proj_dask_fun(testx,testy)
#print(test)